# AKPIK Datathon 2023

Welcome to our Datathon! This Jupyter notebook loads the data for you and takes out a prior evaluation of your solutions. You can also study the data and build your solutions inside this notebook.

**The data** consists of tiny grayscale images (28x28 pixels), which need to be classified into 10 classes. Below, we will take a look at some of these images.

**Your goal** is to find those 10k training examples, from which the best classifier can be trained. The classifier is a fixed 3-layer neural network that is already implemented below. You are given a large pool of training data to select your 10k examples from. You are also given the features of the test set, but not the corresponding labels! Can you figure out the best training examples?

## Basics: data loading and evaluation

The following code implements our evaluation and downloads the data.

In [ ]:
import numpy as np
import tensorflow as tf
from datetime import datetime, timezone

def evaluate(X_trn, y_trn, X_val, y_val, n_trials=10):
    """Evaluate the fixed model with given training and validation sets.
    
    The final evaluation pipeline of the Datathon is based on precisely the same function,
    but employs the true testing data (X_tst, y_tst), of which the labels y_tst remain
    undisclosed to you."""
    print("Evaluating the model...")
    performances = np.zeros(n_trials)
    for i in range(n_trials): # evaluate the training procedure 10 times
        model = tf.keras.Sequential([ # we use a simple, fully-connected, 3-layer neural network
            tf.keras.layers.Flatten(input_shape=(28, 28)), # inputs have 28x28 pixels
            tf.keras.layers.Dense(256, activation="relu"), # ReLU activation with 256 hidden units
            tf.keras.layers.Dense(10) # the output are 10 class probabilities
        ])
        model.compile( # we will optimize for accuracy, using the standard ADAM optimizer
            optimizer = "adam",
            loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
            metrics = ["accuracy"]
        )
        model.fit(X_trn, y_trn, epochs=10, validation_split=.1, verbose=0) # train the neural network
        performances[i] = model.evaluate(X_val, y_val, verbose=0)[1] # evaluate the neural network
        print(f"Testing accuracy [{i+1:02d}/{len(performances)}]: {performances[i]:.4f}")
    print("Average accuracy:", np.mean(performances), "+-", np.std(performances))

def download_data():
    """Download the training features, the training labels, and the testing features."""
    import os
    import urllib.request
    for data_path in ["X_trn.npy", "y_trn.npy", "X_tst.npy"]:
        if not os.path.isfile(data_path):
            url = "https://tu-dortmund.sciebo.de/s/7Bespy29gx955hQ/download?files=" + data_path
            print(f"Downloading {data_path} from {url}")
            urllib.request.urlretrieve(url, data_path)
    print("All data is available now.")
    
    
    
def export_submission(train_indices, group_name):
    """
    Export your submission data
    
    Parameters
    ----------
    train_indices : np.ndarray[dtype=int]
        The 10000 indices of your selected training events
    group_name : str
        the name of your group, should not contain spaces
    """
    if train_indices.shape != (10000, ):
        raise ValueError("You have to submit the indices of 10000 training events")
    
    timestamp = datetime.now(timezone.utc).isoformat()
    np.save(f"submission_{group_name}_{timestamp}.npy", train_indices)

download_data() # execute the function that downloads the data

## A simple baseline solution

How well would a random selection of 10k training examples work? The following code loads the data, selects the first 10k examples from the training pool, and evaluates this selection on the remaining data. It also stores this solution in a file - you should do the same with your solutions, to send them to the organizers of this Datathon for the final evaluation.

In [ ]:
print("Loading the data...")
X_trn = np.load("X_trn.npy") # this is a numpy matrix of the training features
y_trn = np.load("y_trn.npy") # this is a numpy array of the training labels
X_tst = np.load("X_tst.npy") # this is a numpy matrix of the testing features

# baseline: select the first 10k examples
i_trn = np.arange(10_000)

# use all remaining data for validation
#
# hint: which data might be the most appropriate for validation?
i_val = np.arange(10_000, len(y_trn))

# evaluate the baseline
evaluate(
    X_trn[i_trn], y_trn[i_trn], # use the first 10k examples for training...
    X_trn[i_val], y_trn[i_val] # ...and all remaining examples for validation.
)

# store your solution in a file that you can send to the organizers!
np.save("solution_N_of_team_XYZ.npy", i_trn) # always use this technique to store your solution

Almost 60% of accuracy does not seem too bad for a 10-class classification task (random guessing instead of machine learning would only give us about 10% of accuracy here).

However, you will find that this simple baseline will only achieve an accuracy of 42.5% on the actual testing set. Can you find out why the performance drops?

## Taking a look at the data

You can use any tool you like to select your 10k training instances. Your first step in this process, however, should be to take a look at what the data represents. The following code plots the first 25 images of the training data with their labels.

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(X_trn[i], cmap=plt.cm.binary)
    plt.xlabel(y_trn[i])
plt.show()

Looks like fashion! Let's also check what the class labels might represent. The following code plots the first 10 instances of class `9`.

In [ ]:
X_9 = X_trn[y_trn == 9] # select all instances of class 9

plt.figure(figsize=(10,5))
for i in range(15):
    plt.subplot(3,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(X_9[i], cmap=plt.cm.binary)
plt.show()

Most of these examples are boots!

It is now your goal to find out, which training examples are best suited to train a machine learning model that performs well on the testing data `(X_tst, y_tst)`, without knowing `y_tst`.

Good luck!

In [ ]:
# proceed with any code you like

In [ ]:
# base guess, just use the first 10000 events
train_indices = np.arange(10000)

In [ ]:
# export your training indices for submission
export_submission(
    train_indices=train_indices,
    group_name="insert_group_name",
)